# DLC for WBFM analysis: chip setup

This workflow requires three preparation steps:

- Preprocessing from .stk to a large ome.tiff file, currently using FIJI
- Conversion of a slice to .avi, currently using Python
- Manual labeling with the DLC gui, on a local machine
    - Then the created project should be copied to the cluster where this training is run



## Load locally-labeled project


In [1]:
%env DLClight=True
import deeplabcut
import os
from DLC_for_WBFM.utils.training_data.training_settings_utils import *

env: DLClight=True
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [2]:

# Copied from the local machine
project_folder = r"C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d-Charlie-2020-11-02"
# project_folder = '/users/charles.fieseler/DLC_for_WBFM/DLC_Projects/Chip_with_WBFM_hardware_z8-Charlie-2020-10-12'
path_config_file = os.path.join(project_folder, 'config.yaml')


## IF ON CLUSTER: Update the path files to be Unix, not Windows

Notes on how to do this programmatically:
https://github.com/DeepLabCut/DeepLabCut/blob/master/docs/maDLC_AdvUserGuide.md

In [3]:
if os.name is not 'nt':
    video_fname = os.path.join(project_folder, 'videos/27082020_trial2_HEAD_500frames_mcherry_slice8.avi')
    # video_fname = '/users/charles.fieseler/shared_projects/wbfm/dat/immobilized_wbfm_hardware/27082020_trial2_dual1_HEAD_500frames_mcherry_slice8.avi'
    edits = {'project_path': project_folder,
            'video_sets': {video_fname : {'crop':[0, 332, 0, 132]}}}

    deeplabcut.auxiliaryfunctions.edit_config(path_config_file, edits);
else:
    print("Already on Windows machine; no updating required")

Already on Windows machine; no updating required


## Create a training dataset

This function generates the training data information for network training based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles if they want to benchmark the performance (typcailly, 1 is what you will set, so you pass nothing!). 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**. For most all use cases we have seen, the defaults are perfectly fine.

Now it is the time to start training the network!

In [4]:
deeplabcut.create_training_dataset(path_config_file, augmenter_type='imgaug')
#remember, there are several networks you can pick, the default is resnet-50!

C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d-Charlie-2020-11-02\training-datasets\iteration-0\UnaugmentedDataSet_wbfm_3dNov2  already exists!
C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d-Charlie-2020-11-02\dlc-models\iteration-0\wbfm_3dNov2-trainset95shuffle1  already exists!
C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d-Charlie-2020-11-02\dlc-models\iteration-0\wbfm_3dNov2-trainset95shuffle1/train  already exists!
C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d-Charlie-2020-11-02\dlc-models\iteration-0\wbfm_3dNov2-trainset95shuffle1/test  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


[(0.95,
  1,
  (array([29, 24,  7,  3,  2, 15, 28,  1, 16, 10,  4, 23, 13,  6,  8, 26,  9,
          14, 20, 11,  5, 22, 25, 17, 21, 19, 12,  0]),
   array([27, 18])))]

In [5]:
# Update the training files
update_pose_cfg(path_config_file)

Finished updating training config file


## Start training:

This function trains the network for a specific shuffle of the training dataset. 

In [3]:
deeplabcut.train_network(path_config_file, displayiters=100, saveiters=10000)

Config:
{'all_joints': [[0], [1], [2], [3], [4], [5], [6], [7]],
 'all_joints_names': ['Unknown',
                      'REV1',
                      'REV2',
                      'REV3',
                      'SMDD1',
                      'SMDD2',
                      'URYV',
                      'URYD'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_wbfm_3dNov2\\wbfm_3d_Charlie95shuffle1.mat',
 'dataset_type': 'imgaug',
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\charles.fieseler\\Anaconda3\\envs\\DLC-GPU\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev':

Starting with imgaug pose-dataset loader.
Batch Size is 1
Initializing ResNet
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Loading ImageNet-pretrained resnet_50
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\charles.fieseler\Anaconda3\envs\DLC-GPU\lib\site-packages\deeplabcut\pose_estimation_tensorflow\models\pretrained\resnet_v1_50.ckpt
Display_iters overwritten as 100
Save_iters overwritten as 10000
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'C:\\Users\\charles.fieseler\\Documents\\Current_work\\DLC_for_WBFM\\DLC_Projects\\wbfm_3d-Charlie-2020-11-02\\dlc-models\\iteration-0\\wbfm_3dNov2-trainset95shuffle1\\train\\snapshot', 'log_dir': 'l

iteration: 100 loss: 0.0445 lr: 0.02
iteration: 200 loss: 0.0233 lr: 0.02
iteration: 300 loss: 0.0227 lr: 0.02
iteration: 400 loss: 0.0222 lr: 0.02
iteration: 500 loss: 0.0206 lr: 0.02
iteration: 600 loss: 0.0191 lr: 0.02
iteration: 700 loss: 0.0186 lr: 0.02
iteration: 800 loss: 0.0165 lr: 0.02
iteration: 900 loss: 0.0151 lr: 0.02
iteration: 1000 loss: 0.0159 lr: 0.02
iteration: 1100 loss: 0.0128 lr: 0.005
iteration: 1200 loss: 0.0126 lr: 0.005
iteration: 1300 loss: 0.0121 lr: 0.005
iteration: 1400 loss: 0.0123 lr: 0.005
iteration: 1500 loss: 0.0118 lr: 0.005
iteration: 1600 loss: 0.0117 lr: 0.005
iteration: 1700 loss: 0.0110 lr: 0.005
iteration: 1800 loss: 0.0102 lr: 0.005
iteration: 1900 loss: 0.0116 lr: 0.005
iteration: 2000 loss: 0.0105 lr: 0.005
iteration: 2100 loss: 0.0100 lr: 0.005
iteration: 2200 loss: 0.0106 lr: 0.005
iteration: 2300 loss: 0.0096 lr: 0.005
iteration: 2400 loss: 0.0097 lr: 0.005
iteration: 2500 loss: 0.0097 lr: 0.005
iteration: 2600 loss: 0.0100 lr: 0.005
itera

iteration: 20900 loss: 0.0043 lr: 0.001
iteration: 21000 loss: 0.0041 lr: 0.001
iteration: 21100 loss: 0.0040 lr: 0.001
iteration: 21200 loss: 0.0040 lr: 0.001
iteration: 21300 loss: 0.0042 lr: 0.001
iteration: 21400 loss: 0.0042 lr: 0.001
iteration: 21500 loss: 0.0037 lr: 0.001
iteration: 21600 loss: 0.0042 lr: 0.001
iteration: 21700 loss: 0.0039 lr: 0.001
iteration: 21800 loss: 0.0039 lr: 0.001
iteration: 21900 loss: 0.0041 lr: 0.001
iteration: 22000 loss: 0.0041 lr: 0.001
iteration: 22100 loss: 0.0042 lr: 0.001
iteration: 22200 loss: 0.0038 lr: 0.001
iteration: 22300 loss: 0.0042 lr: 0.001
iteration: 22400 loss: 0.0042 lr: 0.001
iteration: 22500 loss: 0.0039 lr: 0.001
iteration: 22600 loss: 0.0039 lr: 0.001
iteration: 22700 loss: 0.0040 lr: 0.001
iteration: 22800 loss: 0.0038 lr: 0.001
iteration: 22900 loss: 0.0040 lr: 0.001
iteration: 23000 loss: 0.0042 lr: 0.001
iteration: 23100 loss: 0.0040 lr: 0.001
iteration: 23200 loss: 0.0041 lr: 0.001
iteration: 23300 loss: 0.0039 lr: 0.001


iteration: 41400 loss: 0.0033 lr: 0.001
iteration: 41500 loss: 0.0035 lr: 0.001
iteration: 41600 loss: 0.0036 lr: 0.001
iteration: 41700 loss: 0.0033 lr: 0.001
iteration: 41800 loss: 0.0035 lr: 0.001
iteration: 41900 loss: 0.0034 lr: 0.001
iteration: 42000 loss: 0.0033 lr: 0.001
iteration: 42100 loss: 0.0033 lr: 0.001
iteration: 42200 loss: 0.0033 lr: 0.001
iteration: 42300 loss: 0.0034 lr: 0.001
iteration: 42400 loss: 0.0032 lr: 0.001
iteration: 42500 loss: 0.0035 lr: 0.001
iteration: 42600 loss: 0.0034 lr: 0.001
iteration: 42700 loss: 0.0034 lr: 0.001
iteration: 42800 loss: 0.0033 lr: 0.001
iteration: 42900 loss: 0.0034 lr: 0.001
iteration: 43000 loss: 0.0034 lr: 0.001
iteration: 43100 loss: 0.0034 lr: 0.001
iteration: 43200 loss: 0.0034 lr: 0.001
iteration: 43300 loss: 0.0034 lr: 0.001
iteration: 43400 loss: 0.0035 lr: 0.001
iteration: 43500 loss: 0.0034 lr: 0.001
iteration: 43600 loss: 0.0035 lr: 0.001
iteration: 43700 loss: 0.0033 lr: 0.001
iteration: 43800 loss: 0.0033 lr: 0.001


...

iteration: 141100 loss: 0.0023 lr: 0.001
iteration: 141200 loss: 0.0024 lr: 0.001
iteration: 141300 loss: 0.0023 lr: 0.001
iteration: 141400 loss: 0.0023 lr: 0.001
iteration: 141500 loss: 0.0024 lr: 0.001
iteration: 141600 loss: 0.0024 lr: 0.001
iteration: 141700 loss: 0.0024 lr: 0.001
iteration: 141800 loss: 0.0025 lr: 0.001
iteration: 141900 loss: 0.0025 lr: 0.001
iteration: 142000 loss: 0.0025 lr: 0.001
iteration: 142100 loss: 0.0025 lr: 0.001
iteration: 142200 loss: 0.0024 lr: 0.001
iteration: 142300 loss: 0.0024 lr: 0.001
iteration: 142400 loss: 0.0023 lr: 0.001
iteration: 142500 loss: 0.0024 lr: 0.001
iteration: 142600 loss: 0.0025 lr: 0.001
iteration: 142700 loss: 0.0024 lr: 0.001
iteration: 142800 loss: 0.0023 lr: 0.001
iteration: 142900 loss: 0.0026 lr: 0.001
iteration: 143000 loss: 0.0024 lr: 0.001
iteration: 143100 loss: 0.0025 lr: 0.001
iteration: 143200 loss: 0.0024 lr: 0.001
iteration: 143300 loss: 0.0025 lr: 0.001
iteration: 143400 loss: 0.0023 lr: 0.001
iteration: 14350

iteration: 161100 loss: 0.0022 lr: 0.001
iteration: 161200 loss: 0.0023 lr: 0.001
iteration: 161300 loss: 0.0023 lr: 0.001
iteration: 161400 loss: 0.0023 lr: 0.001
iteration: 161500 loss: 0.0023 lr: 0.001
iteration: 161600 loss: 0.0022 lr: 0.001
iteration: 161700 loss: 0.0023 lr: 0.001
iteration: 161800 loss: 0.0023 lr: 0.001
iteration: 161900 loss: 0.0022 lr: 0.001
iteration: 162000 loss: 0.0022 lr: 0.001
iteration: 162100 loss: 0.0022 lr: 0.001
iteration: 162200 loss: 0.0021 lr: 0.001
iteration: 162300 loss: 0.0022 lr: 0.001
iteration: 162400 loss: 0.0023 lr: 0.001
iteration: 162500 loss: 0.0024 lr: 0.001
iteration: 162600 loss: 0.0023 lr: 0.001
iteration: 162700 loss: 0.0023 lr: 0.001
iteration: 162800 loss: 0.0022 lr: 0.001
iteration: 162900 loss: 0.0024 lr: 0.001
iteration: 163000 loss: 0.0021 lr: 0.001
iteration: 163100 loss: 0.0023 lr: 0.001
iteration: 163200 loss: 0.0023 lr: 0.001
iteration: 163300 loss: 0.0021 lr: 0.001
iteration: 163400 loss: 0.0022 lr: 0.001
iteration: 16350

iteration: 181100 loss: 0.0023 lr: 0.001
iteration: 181200 loss: 0.0022 lr: 0.001
iteration: 181300 loss: 0.0023 lr: 0.001
iteration: 181400 loss: 0.0022 lr: 0.001
iteration: 181500 loss: 0.0022 lr: 0.001
iteration: 181600 loss: 0.0021 lr: 0.001
iteration: 181700 loss: 0.0021 lr: 0.001
iteration: 181800 loss: 0.0022 lr: 0.001
iteration: 181900 loss: 0.0020 lr: 0.001
iteration: 182000 loss: 0.0022 lr: 0.001
iteration: 182100 loss: 0.0021 lr: 0.001
iteration: 182200 loss: 0.0022 lr: 0.001
iteration: 182300 loss: 0.0022 lr: 0.001
iteration: 182400 loss: 0.0022 lr: 0.001
iteration: 182500 loss: 0.0022 lr: 0.001
iteration: 182600 loss: 0.0020 lr: 0.001
iteration: 182700 loss: 0.0022 lr: 0.001
iteration: 182800 loss: 0.0020 lr: 0.001
iteration: 182900 loss: 0.0021 lr: 0.001
iteration: 183000 loss: 0.0023 lr: 0.001
iteration: 183100 loss: 0.0023 lr: 0.001
iteration: 183200 loss: 0.0022 lr: 0.001
iteration: 183300 loss: 0.0023 lr: 0.001
iteration: 183400 loss: 0.0021 lr: 0.001
iteration: 18350

The network is now trained and ready to evaluate. Use the function 'evaluate_network' to evaluate the network.


Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\charles.fieseler\Anaconda3\envs\DLC-GPU\lib\site-packages\tensorflow\python\client\session.py", line 1334, in _do_call
    return fn(*args)
  File "C:\Users\charles.fieseler\Anaconda3\envs\DLC-GPU\lib\site-packages\tensorflow\python\client\session.py", line 1319, in _run_fn
    options, feed_dict, fetch_list, target_list, run_metadata)
  File "C:\Users\charles.fieseler\Anaconda3\envs\DLC-GPU\lib\site-packages\tensorflow\python\client\session.py", line 1407, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.CancelledError: Enqueue operation was cancelled
	 [[{{node fifo_queue_enqueue}}]]

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\charles.fieseler\Anaconda3\envs\DLC-GPU\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\charles.fieseler\Anaconda3\envs\DLC-GPU\lib\thread

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [4]:
deeplabcut.evaluate_network(path_config_file, plotting=True)

Note: NumExpr detected 56 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.
Config:
{'all_joints': [[0], [1], [2], [3], [4], [5], [6], [7]],
 'all_joints_names': ['Unknown',
                      'REV1',
                      'REV2',
                      'REV3',
                      'SMDD1',
                      'SMDD2',
                      'URYV',
                      'URYD'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_wbfm_3dNov2\\wbfm_3d_Charlie95shuffle1.mat',
 'dataset_type': 'imgaug',
 'deconvolutionstride': 2,
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\charles.fieseler\\Anaconda3\\envs\\DLC-GPU\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'in

Running  DLC_resnet50_wbfm_3dNov2shuffle1_200000  with # of trainingiterations: 200000
Initializing ResNet
INFO:tensorflow:Restoring parameters from C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d-Charlie-2020-11-02\dlc-models\iteration-0\wbfm_3dNov2-trainset95shuffle1\train\snapshot-200000


0it [00:00, ?it/s]

Analyzing data...


30it [00:02, 12.15it/s]
  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

Done and results stored for snapshot:  snapshot-200000
Results for 200000  training iterations: 95 1 train error: 2.52 pixels. Test error: 2.74  pixels.
With pcutoff of 0.6  train error: 2.52 pixels. Test error: 2.74 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
Plotting...


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:09<00:00,  3.09it/s]

The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)


## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [5]:
# Get video in default folder
cfg = deeplabcut.auxiliaryfunctions.read_config(path_config_file)
video_fname = [i for i in cfg['video_sets'].keys()] # Assume one video

In [6]:
video_fname

['C:\\Users\\charles.fieseler\\Documents\\Current_work\\DLC_for_WBFM\\DLC_Projects\\wbfm_3d-Charlie-2020-11-02\\videos\\test2020-10-22_mcherry_fr1000_sl10_15.avi']

In [7]:

deeplabcut.analyze_videos(path_config_file,video_fname, videotype='.avi')

Config:
{'all_joints': [[0], [1], [2], [3], [4], [5], [6], [7]],
 'all_joints_names': ['Unknown',
                      'REV1',
                      'REV2',
                      'REV3',
                      'SMDD1',
                      'SMDD2',
                      'URYV',
                      'URYD'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_wbfm_3dNov2\\wbfm_3d_Charlie95shuffle1.mat',
 'dataset_type': 'imgaug',
 'deconvolutionstride': 2,
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\charles.fieseler\\Anaconda3\\envs\\DLC-GPU\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weig

Using snapshot-200000 for model C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d-Charlie-2020-11-02\dlc-models\iteration-0\wbfm_3dNov2-trainset95shuffle1
Initializing ResNet
INFO:tensorflow:Restoring parameters from C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d-Charlie-2020-11-02\dlc-models\iteration-0\wbfm_3dNov2-trainset95shuffle1\train\snapshot-200000


  0%|                                                                                         | 0/1001 [00:00<?, ?it/s]

Starting to analyze %  C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d-Charlie-2020-11-02\videos\test2020-10-22_mcherry_fr1000_sl10_15.avi
Loading  C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d-Charlie-2020-11-02\videos\test2020-10-22_mcherry_fr1000_sl10_15.avi
Duration of video [s]:  100.1 , recorded with  10.0 fps!
Overall # of frames:  1001  found with (before cropping) frame dimensions:  900 700
Starting to extract posture


1010it [00:39, 24.06it/s]                                                                                              

Detected frames:  1001


1010it [00:39, 25.59it/s]

Saving results in C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d-Charlie-2020-11-02\videos...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract any outlier frames!


'DLC_resnet50_wbfm_3dNov2shuffle1_200000'

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

THIS HAS MANY FUN OPTIONS! 

``deeplabcut.create_labeled_video(config, videos, videotype='avi', shuffle=1, trainingsetindex=0, filtered=False, save_frames=False, Frames2plot=None, delete=False, displayedbodyparts='all', codec='mp4v', outputframerate=None, destfolder=None, draw_skeleton=False, trailpoints=0, displaycropped=False)``

So please check:

In [9]:
# video_fname = os.path.join(project_folder, 'videos/27082020_trial2_HEAD_500frames_mcherry_slice8.avi')

deeplabcut.create_labeled_video(path_config_file,video_fname)

Starting %  C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d-Charlie-2020-11-02\videos ['C:\\Users\\charles.fieseler\\Documents\\Current_work\\DLC_for_WBFM\\DLC_Projects\\wbfm_3d-Charlie-2020-11-02\\videos\\test2020-10-22_mcherry_fr1000_sl10_15.avi']
Labeled video already created.


## Extract outlier frames [optional step]

This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. This step has many options, so please look at:

In [27]:
deeplabcut.extract_outlier_frames(path_config_file,video_fname) #pass a specific video

Method  jump  found  983  putative outlier frames.
Do you want to proceed with extracting  10  of those?
If this list is very large, perhaps consider changing the paramters (start, stop, epsilon, comparisonbodyparts) or use a different method.
yes/noyes


40it [00:00, 201.05it/s]

Frames from video mcherry_frames1000_slice17_22  already extracted (more will be added)!
Loading video...
Duration of video [s]:  100.1 , recorded @  10.0 fps!
Overall # of frames:  1001 with (cropped) frame dimensions: 
Kmeans-quantization based extracting of frames from 0.0  seconds to 100.1  seconds.
Extracting and downsampling... 983  frames from the video.


983it [00:05, 189.81it/s]


Kmeans clustering ... (this might take a while)
Let's select frames indices: [197, 145, 389, 767, 557, 708, 104, 188, 390, 237]


C:\Users\charles.fieseler\Anaconda3\envs\DLC-GPU\lib\site-packages\deeplabcut\refine_training_dataset\outlier_frames.py:463: UserWarning: C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d-Charlie-2020-10-21\labeled-data\mcherry_frames1000_slice17_22\img0197.png is a low contrast image
  io.imsave(imagename1,image)
C:\Users\charles.fieseler\Anaconda3\envs\DLC-GPU\lib\site-packages\deeplabcut\refine_training_dataset\outlier_frames.py:463: UserWarning: C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d-Charlie-2020-10-21\labeled-data\mcherry_frames1000_slice17_22\img0389.png is a low contrast image
  io.imsave(imagename1,image)
C:\Users\charles.fieseler\Anaconda3\envs\DLC-GPU\lib\site-packages\deeplabcut\refine_training_dataset\outlier_frames.py:463: UserWarning: C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d-Charlie-2020-10-21\labeled-data\mcherry_frames1000_slice17_22\img0767.png is a lo

New video was added to the project! Use the function 'extract_frames' to select frames for labeling.
The outlier frames are extracted. They are stored in the subdirectory labeled-data\mcherry_frames1000_slice17_22.
Once you extracted frames for all videos, use 'refine_labels' to manually correct the labels.


## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset. 

In [ ]:
%gui wx
deeplabcut.refine_training_dataset.refine_labels(path_config_file)

Closing... The refined labels are stored in a subdirectory under labeled-data. Use the function 'merge_datasets' to augment the training dataset, and then re-train a network using create_training_dataset followed by train_network!


**NOTE:** Afterwards, if you want to look at the adjusted frames, you can load them in the main GUI by running: ``deeplabcut.label_frames(path_config_file)``

(you can add a new "cell" below to add this code!)

#### Once all folders are relabeled, check the labels again! If you are not happy, adjust them in the main GUI:

``deeplabcut.label_frames(path_config_file)``

Check Labels:

``deeplabcut.check_labels(path_config_file)``

In [ ]:
#NOW, merge this with your original data:

deeplabcut.merge_datasets(path_config_file)

## Create a new iteration of training dataset [optional step]
Following the refinement of labels and appending them to the original dataset, this creates a new iteration of training dataset. This is automatically set in the config.yaml file, so let's get training!

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)